In [1]:
import requests 
import time
import json
from bs4 import  BeautifulSoup
import pandas as pd

In [2]:
def get_proxy():
    return requests.get("http://127.0.0.1:5010/get/").json()

def delete_proxy(proxy):
    requests.get("http://127.0.0.1:5010/delete/?proxy={}".format(proxy))

def getHtml(url):
    retry_count = 10
    proxy = get_proxy().get("proxy")
    while retry_count > 0:
        try:
            # 使用代理访问
            html = requests.get(url, proxies={"http": "http://{}".format(proxy)})
            return html
        except Exception:
            retry_count -= 1
            # 删除代理池中代理
            delete_proxy(proxy)
    print('无可用代理')
    return None

In [3]:
with open('place_info_contain_location.txt', 'r') as f:   
    lines = f.readlines() 
lines_str = str(lines).replace('\'', '"')[2:-2]
city_url_info = json.loads(lines_str)

In [4]:
'''
    获取区域的所有景点
'''
def get_place_url(url):
    response = getHtml(url)
    location_place_url_soup=BeautifulSoup(response.text,'lxml')
    location_place_url_data = location_place_url_soup.select('#tiles > li')

    place_info_list = []

    for item in location_place_url_data:

        try:
            place_info={}
            place_info['href'] = item.a.get('href')

            item2 = item.select('li > div.bar')[0]
            place_info['name'] = item2.attrs['m']
            place_info['ever'] = item2.select('a.ever > span')[0].get_text()
            place_info['want'] = item2.select('a.never > span')[0].get_text()

            place_info_list.append(place_info)

        except Exception as e:
            print(e)
    time.sleep(2)
    return place_info_list

In [5]:
'''
    获取景点信息
'''
def get_place_info(url):

    place_response = getHtml(url)
    place_soup=BeautifulSoup(place_response.text,'html5lib')
    place_header_data = place_soup.select('#jdleft > div.info > div > div.hd > ul > li')
    place_data = place_soup.select('#jdleft > div.info > div > div > div.item')

    place_info_item = {}
    for i in range(len(place_header_data)):

        if place_header_data[i].get_text() in ['学校介绍', '景区介绍', '旅游攻略', '怎么去', '门票']:
            # print(place_header_data[i].get_text())
            # print(place_data[i])
            item_header = place_data[i].find_all('h2')
            # print(item_header)
            place_data_text = place_data[i].get_text()
            
            if place_header_data[i].get_text()=='旅游攻略':
                lv_gl_text_data = place_data[i].select('ul')
                if len(lv_gl_text_data) > 0:
                    lv_gl_text = lv_gl_text_data[0].get_text()
                    place_data_text = place_data_text.replace(lv_gl_text, '')
            
            place_data_text = place_data_text.replace('信息纠错', '')
            place_data_text = place_data_text.replace('景区的卫星地图', '')

            

            place_info_item[place_header_data[i].get_text()] = {}

            for j in range(len(item_header)):
                current_header = item_header[j].get_text()[:-1]
                current_text = place_data_text[place_data_text.find(current_header)+len(current_header)+1:]

                if j < len(item_header)-1:
                    next_header = item_header[j+1].get_text()
                    current_text = current_text[:current_text.find(next_header)]
                
                if current_header not in ['景区游览图', '学校地图', '景区地图', '驴友游记']:
                    place_info_item[place_header_data[i].get_text()][current_header] = current_text.replace('　　', '').replace(' ', '').split('\n')
    time.sleep(2)
    
    return place_info_item



In [6]:
location_error_list = []
error_list = []
base_url = 'https://www.meet99.com'
rlt = {'name':[], 'province': [], 'city': [], 'location': [], 'desc': [], 'want': [], 'has_been': [], 'strategy': [], 'route': [], 'ticket_info': [], 'open_time': []}

for province_key in city_url_info.keys():
    if province_key not in ['北京市', '天津市', '上海市', '重庆市', '香港', '澳门'] and province_key in ['山东省', '河南省']:
        for city_key in city_url_info[province_key]['children'].keys():
            # print(city_key)
            
            if city_key not in ['济南市', '青岛市', '淄博市', '枣庄市', '东营市', '烟台市', '潍坊市', '济宁市', '泰安市'] and 'children' in city_url_info[province_key]['children'][city_key] and len(city_url_info[province_key]['children'][city_key]['children'].keys()) > 0 :
                
                for location_key in city_url_info[province_key]['children'][city_key]['children'].keys():
                    location_info = city_url_info[province_key]['children'][city_key]['children'][location_key]
                    location_url = base_url + location_info['link']
                    print(location_key)
                    print(location_url)
                    location_place_info_list = []
                    try:
                        location_place_info_list = get_place_url(location_url)

                    except Exception as e:
                        print('获取区域内的景点出现异常,异常原因:', e)
                        print(location_info)
                        location_error_info = {}
                        location_error_info['province_key'] = province_key
                        location_error_info['city_key'] = city_key
                        location_error_info['location_name'] = location_key
                        location_error_info['location_info'] = location_info
                        location_error_list.append(location_error_info)


                    for place_item in location_place_info_list:
                        try:
                            print(place_item)
                            place_url = base_url + place_item['href']
                            place_info = get_place_info(place_url)

                            name = place_item['name']
                            
                            province = province_key
                            city = city_key
                            location = location_key
                            desc = {}
                            if '景区介绍' in place_info.keys():
                                desc = place_info['景区介绍']
                            elif '学校介绍' in place_info.keys():
                                desc = place_info['学校介绍']

                            want = place_item['want']
                            has_been = place_item['ever']

                            strategy = {}
                            if '旅游攻略' in place_info.keys():
                                strategy = place_info['旅游攻略']

                            route = {}
                            if '怎么去' in place_info.keys():
                                route = place_info['怎么去']

                            ticket_info = {}
                            open_time = {}
                            if '门票' in place_info.keys():
                                for key in place_info['门票']:
                                    if key not in ['景区开放时间']:
                                        ticket_info[key] = place_info['门票'][key]


                                if '景区开放时间' in place_info['门票'].keys():
                                    open_time = place_info['门票']['景区开放时间']
                            
                            ticket = ticket_info

                            rlt['name'].append(name)
                            rlt['province'].append(province)
                            rlt['city'].append(city)
                            rlt['location'].append(location)
                            rlt['desc'].append(desc)
                            rlt['want'].append(want)
                            rlt['has_been'].append(has_been)
                            rlt['strategy'].append(strategy)
                            rlt['route'].append(route)
                            rlt['ticket_info'].append(ticket_info)
                            rlt['open_time'].append(open_time)
                            

                        except Exception as e:
                            print('以下景点出现异常,异常原因:', e)
                            print(place_item)
                            error_info = {}
                            error_info['province_key'] = province_key
                            error_info['city_key'] = city_key
                            error_info['location_name'] = location_key
                            error_info['place_info'] = place_item
                            error_list.append(error_info)  

                time.sleep(5)

                    


环翠区
https://www.meet99.com/lvyou-huancui.html


'NoneType' object has no attribute 'get'
{'href': '/jingdian-liugongdao.html', 'name': '威海刘公岛', 'ever': '4430', 'want': '1158'}
{'href': '/jingdian-dingyuanjian.html', 'name': '威海定远舰', 'ever': '750', 'want': '118'}
{'href': '/jingdian-jiawuzhanzheng.html', 'name': '威海中国甲午战争博物馆', 'ever': '488', 'want': '32'}
{'href': '/jingdian-huaxiacheng.html', 'name': '威海华夏城', 'ever': '270', 'want': '74'}
{'href': '/jingdian-xianguding.html', 'name': '威海仙姑顶旅游区', 'ever': '260', 'want': '80'}
{'href': '/jingdian-whgjhsyc.html', 'name': '威海林海湾国际海水浴场', 'ever': '305', 'want': '67'}
{'href': '/jingdian-weihaihuancuidao.html', 'name': '威海环翠楼', 'ever': '192', 'want': '24'}
{'href': '/jingdian-qiaoxianghao.html', 'name': '威海海滨生态公园风景带侨乡号游轮', 'ever': '1', 'want': '0'}
{'href': '/jingdian-whg.html', 'name': '山东威海幸福公园', 'ever': '296', 'want': '16'}
{'href': '/jingdian-maozedongxiangzhang.html', 'name': '威海毛泽东像章珍藏馆', 'ever': '0', 'want': '0'}
{'href': '/jingdian-liugongdaobly.html', 'name': '威海刘公岛博览园', 'ever': '18

KeyboardInterrupt: 

In [7]:
location_error_list

[]

In [8]:
error_list

[]

In [9]:
rlt_df = pd.DataFrame(rlt)

In [10]:
rlt_df

,name,province,city,location,desc,want,has_been,strategy,route,ticket_info,open_time
0,威海刘公岛,山东省,威海市,环翠区,{'景区简介': ['刘公岛自然风光优美，人文景观独特，素有“东隅屏藩”、“海上仙山”和“世...,1158,4430,{'旅游攻略': ['亲临中国近代第一支海军枣清政府北洋水师的诞生地，这也是中日甲午战争的古...,"{'景区位置': ['中国>山东省>威海市>环翠区>嵩山街道'], '到达方式': ['威海...","{'刘公岛门票价格': ['景区门票：122元/人（含景点门票和往返船票）', '索道：双城...",[刘公岛轮渡游船时刻：旺季7:00-16:308分钟10：00-17:30；淡季7:30-1...
1,威海定远舰,山东省,威海市,环翠区,{'景区简介': ['威海定远舰曾是19世纪工业文明的产物，因武备强大、防护完善，一度被誉为...,118,750,"{'最佳游览时间': ['夏季。'], '购物推荐': ['(当地特产)港西无花果、锡镶茶具...","{'景区位置': ['中国>山东省>威海市>环翠区>嵩山街道'], '到达方式': ['游客...",{'定远舰门票价格': ['票价50元。']},[8:00-16:30]
2,威海中国甲午战争博物馆,山东省,威海市,环翠区,{'景区简介': ['威海中国甲午战争博物馆以北洋海军和甲午战争为主题，以丰富的历史遗迹和特...,32,488,"{'最佳游览时间': ['四季皆宜'], '购物推荐': ['北洋海军提督署入口处、一进院落...","{'景区位置': ['中国>山东省>威海市>环翠区>嵩山街道'], '到达方式': ['乘车...",{'甲午战争门票价格': ['免费开放']},"[淡季8:30—16:30(16:00停止入场), 旺季7:30—18:00(17:30停止..."
3,威海华夏城,山东省,威海市,环翠区,{'景区简介': ['威海“华夏城”是一处集旅游、休闲、商贸、居住等综合功能于一体的现代化旅...,74,270,"{'旅游攻略': ['1、神游海洋世界表演时间：', 'a.动感影院：10分钟，9：00、1...","{'景区位置': ['中国>山东省>威海市>环翠区>嵩山街道'], '到达方式': ['威海...","{'华夏城门票价格': ['1、华夏城景区：98元', '特殊群体优惠政策如下：', 'A、...","[华夏城景区：7：45—17：00，16：30停止检票。, 游乐园：8：30—17：00，1..."
4,威海仙姑顶旅游区,山东省,威海市,环翠区,{'景区简介': ['威海仙姑顶旅游区被成为“天下第一玉景区”。景区投入2000余吨世界珍稀...,80,260,{'旅游攻略': ['民间根据传说，把农历3月15尊为仙姑的生日，每年从这一天始，仙姑顶下的...,"{'景区位置': ['中国>山东省>威海市>环翠区>嵩山街道'], '到达方式': ['威海...",{'仙姑顶门票价格': ['票价80元。']},[7：30—18：00]
...,...,...,...,...,...,...,...,...,...,...,...
79,临沂鲁商知春湖温泉,山东省,临沂市,河东区,{'景区简介': ['鲁商知春湖温泉是集温泉康养、禅修礼敬、会议会展、主题宴会、休闲游乐等功...,0,0,{'最佳游览时间': ['冬季']},"{'景区位置': ['中国>山东省>临沂市>河东区>朝阳街道'], '到达方式': ['公交...",{'鲁商知春湖温泉门票价格': ['成人138元，1.2米（不含）~1.4米（含）儿童69元...,"[周二至周四14:00-22:00, 周五至周日10:00-22:00, 周一不开放]"
80,沂南诸葛亮文化旅游区,山东省,临沂市,沂南县,{'景区简介': ['沂南诸葛亮文化旅游区以一代智圣诸葛亮的生平为主线，是国内外展示诸葛亮智...,1,20,{'游览路线': ['诸葛亮铜像→诸葛亮宗祠→十里汉街→诸葛草庐→八卦迷宫→智慧桥→圣贤路'...,"{'景区位置': ['中国>山东省>临沂市>沂南县'], '到达方式': ['乘车路线：长途...",{'诸葛亮门票价格': ['免费']},[08：00—18：00]
81,沂南朱家林旅游区,山东省,临沂市,沂南县,{'景区简介': ['朱家林旅游区是山东省唯一的国家级田园综合体，中国乡村旅游创客示范基地，...,0,0,{'最佳游览时间': ['夏秋两季']},"{'景区位置': ['中国>山东省>临沂市>沂南县'], '到达方式': ['双岭高架路→京...",{'朱家林门票价格': ['免费，娱乐项目单独收费。']},"[4月至10月：8:00-19:00, 11月至次年3月：8:30-18:30]"
82,山东郯城郯子公园,山东省,临沂市,郯城县,{'景区简介': ['山东郯城郯子公园为纪念一代国君郯子而兴建，文化内涵丰富，是休闲散步、品...,1,15,{'最佳游览时间': ['四季皆宜']},"{'景区位置': ['中国>山东省>临沂市>郯城县'], '到达方式': ['自驾路线：京沪...",{'郯子公园门票价格': ['免费']},[全天]


In [ ]:
rlt_df.to_csv('山东景点v2.csv', header=True, index=False)